# 转换与处理时间序列数据

In [ ]:
import pandas as pd

对数据进行转换的目的，是为了让数据更适合分析、建模或可视化。不同的转换方式可以揭示隐藏的模式、提高模型性能，或解决数据质量问题。
pandas库继承了Numpy库的datetime库与时间相关模块，提供了六种时间相关的类：

|类名称|说明|
|--|--|
|Timestamp|表示单个时间点，类似于 Python 的 datetime.datetime。用于表示某一具体时间。|
|Period|表示一个时间段（如某月、某季度），而不是具体时间点。|
|Timedelta|表示两个时间点之间的时间差，可用于时间加减运算。|
|DatetimeIndex|时间戳的索引集合，常用于时间序列的索引。支持切片、重采样等操作。|
|PeriodtimeIndex|Period 的集合，适用于周期性数据的索引。|
|TimedeltaIndex|Timedelta 的集合，常用于表示时间间隔序列|


## 转换时间字符串为标准时间

### 转换时间字符串为标准时间

使用to_datetime函数将与时间相关的字符串转换为Timestamp形式

```
pd.to_datetime("2025-10-15")
# 输出: Timestamp('2025-10-15 00:00:00')
```

常用参数说明
| 参数 |说明  | 
|--|--|
| arg | 要转换的对象（字符串、列表、Series、DataFrame） | 
| format | 指定日期格式（如 "%Y-%m-%d"），可加速解析| 
| errors |'raise'（默认）遇错报错，'coerce' 转为 NaT，'ignore' 原样返回 | 
| dayfirst,yearfirst |接受bool；是否将日期中的“日”视为第一位（如英式格式 15/10/2025） | 
| utc | 是否将结果转换为 UTC 时间 | 


In [ ]:
musicdata = pd.read_table('pd_data/musicdata.csv', sep=',', encoding='gbk')

In [ ]:
musicdata

In [ ]:
print(musicdata['date'].dtypes)

In [ ]:
print(musicdata['date'])

In [ ]:
musicdata['dateChange'] = pd.to_datetime(musicdata['date'])

In [ ]:
print(musicdata['dateChange'].dtypes)

In [ ]:
print(musicdata['dateChange'])

### Timestamp的最小时间与最大时间

In [ ]:
print('最小时间为：', pd.Timestamp.min)  # 查询计算机中最早的时间信息
print('最大时间为：', pd.Timestamp.max)  # 查询计算机中最晚的时间信息

In [ ]:
musicdata['dateChange'].min()

In [ ]:
musicdata['dateChange'].max()

In [ ]:
musicdata

### 转换时间字符串为DatetimeIndex与PeriodIndex

DatetimeIndex：精确时间点序列
- 定义：由多个 Timestamp 构成的索引，表示具体的时间点（如某年某月某日某时某分某秒）。
- 适用场景：适合处理高频数据，如每秒、每分钟、每天的传感器数据、股票交易数据等。
- 特点：
    - 支持时间切片、重采样、滚动窗口等操作。
    - 可以进行时间运算，如加减 Timedelta。
    - 精度高，支持纳秒级别。


```
dt_index = pd.date_range(start='2025-01-01', periods=5, freq='D')
```

PeriodIndex：固定周期时间段序列
- 定义：由多个 Period 构成的索引，表示某个时间段（如某月、某季度、某年）。
- 适用场景：适合处理周期性数据，如月度销售额、季度财报、年度统计数据等。
- 特点：
    - 表示的是时间段而非具体时间点。
    - 支持周期运算，如周期偏移、频率转换。
    - 更适合做周期聚合和趋势分析。

```
period_index = pd.period_range(start='2025-01', periods=5, freq='M')
```

如何选择？
| 数据类型 | 推荐索引类型 | 
|-|-|
| 每日或更高频率数据 |  DatetimeIndex| 
|月度、季度数据  | PeriodIndex | 
| 需要精确时间点 | DatetimeIndex | 
| 关注时间段统计 |  PeriodIndex| 






In [ ]:
dateIndex = pd.DatetimeIndex(musicdata['date'])

In [ ]:
type(dateIndex)

In [ ]:
periodIndex = pd.PeriodIndex(musicdata['date'], freq='S')

/tmp/ipykernel_25158/2358369031.py:1: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  periodIndex = pd.PeriodIndex(musicdata['date'], freq='S')

时指定的频率 'S'（大写）已经被标记为“即将废弃”，建议改用小写的 's'。

In [ ]:
type(periodIndex)

## 提取时间序列数据信息

Timestamp 是 Pandas 中用于表示单个时间点的核心类，它是 datetime.datetime 的增强版本，拥有更多功能和属性。下面是它的常用属性和方法，适用于时间序列分析、时间切片、时间运算等场景。

Timestamp 类的常用属性
|属性名  |  说明与示例|
|-|-| 
| year | 年份，如 2025 | 
| month | 月份（1–12） | 
| day |日期（1–31） | 
| hour |  小时（0–23）| 
| minute | 分钟（0–59） | 
| second | 秒（0–59） | 
| microsecond | 微秒（0–999999） | 
| nanosecond | 纳秒（0–999999999） | 
| dayofweek | 星期几（周一为 0，周日为 6） | 
| dayofyear | 一年中的第几天（1–366） | 
| weekweekofyear | 一年中的第几周（已弃用，建议用 isocalendar().week） | 
| quarter | 季度（1–4） | 
| is_month_end | 是否为月末（True/False） | 
| is_month_start | 是否为月初 | 
| is_year_end | 是否为年末 | 
| is_year_start | 是否为年初 | 
|is_leap_year| 是否为闰年 |
| tz | 时区信息 | 
| freq | 时间频率（如果有） | 
| to_pydatetime() | 转换为原生 datetime.datetime 对象


In [ ]:
ts = pd.Timestamp('2025-10-15 10:30:45.123456789')

print(ts.year)          
print(ts.month)          
print(ts.dayofweek)      
print(ts.is_month_end)   
print(ts.to_pydatetime())


tmp/ipykernel_25158/2305970870.py:7: UserWarning: Discarding nonzero nanoseconds in conversion.
  print(ts.to_pydatetime())

这个警告信息说明你在使用 Timestamp.to_pydatetime() 方法时，该时间戳包含纳秒级信息（nanoseconds），但 Python 的原生 datetime.datetime 类型只支持到微秒（microseconds）级别，因此 Pandas 会自动舍弃纳秒部分。


### 提取数据中的年、月、日

In [ ]:
year1 = [i.year for i in musicdata['dateChange']]
month1 = [i.month for i in musicdata['dateChange']]
day1 = [i.day for i in musicdata['dateChange']]

In [ ]:
year1[:5]

In [ ]:
month1[:5]

In [ ]:
day1[:5]

### 提取DatetimeIndex与PeriodIndex中的数据

In [ ]:
dateIndex.weekday[:5]  

In [ ]:
periodIndex.weekday[:5]

## 加减时间数据

Pandas 的Timedelta类是专门用来表示两个时间点之间的时间差，它是时间序列分析中非常重要的工具，尤其适用于时间运算、偏移、间隔计算等场景。

 什么是 Timedelta？
- Timedelta 表示一个时间间隔，比如“5天”、“2小时30分钟”。
- 它是 Pandas 对 Python 原生 datetime.timedelta 的扩展，支持更高精度（纳秒级）和更丰富的操作。

常用属性
|属性名  | 说明 | 
|-|-|
| .days | 间隔的天数 | 
| .seconds | 间隔的秒数（不含天） | 
| .microseconds | 微秒数 | 
| .components | 返回一个结构体，包含天、小时、分钟、秒、毫秒等 | 
| .total_seconds() | 返回总秒数（浮点数） | 


### 实现时间数据的加运算

In [ ]:
time1 = musicdata['dateChange'] + pd.Timedelta(days=1) 

In [ ]:
musicdata['dateChange'][:5]

In [ ]:
time1[:5]

### 实现时间数据的减运算

In [ ]:
timeDelta = musicdata['dateChange'] - pd.to_datetime('2020-1-1')

In [ ]:
timeDelta[:5]


In [ ]:
timeDelta.dtypes

dtype('<m8[ns]') 是 NumPy 和 Pandas 中用于表示时间差（时间间隔）数据的一种数据类型，它对应的是：
timedelta64[ns] 类型
- m8 是 timedelta64 的别名（m 表示“时间间隔”，即 duration）。
- [ns] 表示单位是纳秒（nanoseconds）。
- 所以 dtype('<m8[ns]') 表示：一个以纳秒为单位的时间间隔数据类型。
